{'label': 4,
 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."}

In [9]:

with open("/Users/jagpreetsingh/ML_Projects/text-sql/spider_train.txt", "r", encoding="utf-8") as f:
    dataset = f.read()

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

AttributeError: 'str' object has no attribute 'map'

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [37]:
# # from transformers import AutoModelForSequenceClassification
# import torch

# # model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)
# import sys

# # sys.path.append("../src-llm")
# torch.load("my_model.pt")


In [24]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open("instructions.txt", "r", encoding="utf-8") as f:
    text = f.read()


# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    

## instantiate the model
model = BigramLanguageModel()
# m = model.to(device)

# # print the number of parameters in the model
# print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# # create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# # print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

# for iter in range(max_iters):

#     # every once in a while evaluate the loss on train and val sets
#     if iter % eval_interval == 0 or iter == max_iters - 1:
#         losses = estimate_loss()
#         print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

#     # sample a batch of data
#     xb, yb = get_batch('train')

#     # evaluate the loss
#     logits, loss = model(xb, yb)
#     optimizer.zero_grad(set_to_none=True)
#     loss.backward()
#     optimizer.step()

# # generate from the model
# context = torch.zeros((1, 1), dtype=torch.long, device=device)
# print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


Model's state_dict:
token_embedding_table.weight 	 torch.Size([83, 64])
position_embedding_table.weight 	 torch.Size([32, 64])
blocks.0.sa.heads.0.tril 	 torch.Size([32, 32])
blocks.0.sa.heads.0.key.weight 	 torch.Size([16, 64])
blocks.0.sa.heads.0.query.weight 	 torch.Size([16, 64])
blocks.0.sa.heads.0.value.weight 	 torch.Size([16, 64])
blocks.0.sa.heads.1.tril 	 torch.Size([32, 32])
blocks.0.sa.heads.1.key.weight 	 torch.Size([16, 64])
blocks.0.sa.heads.1.query.weight 	 torch.Size([16, 64])
blocks.0.sa.heads.1.value.weight 	 torch.Size([16, 64])
blocks.0.sa.heads.2.tril 	 torch.Size([32, 32])
blocks.0.sa.heads.2.key.weight 	 torch.Size([16, 64])
blocks.0.sa.heads.2.query.weight 	 torch.Size([16, 64])
blocks.0.sa.heads.2.value.weight 	 torch.Size([16, 64])
blocks.0.sa.heads.3.tril 	 torch.Size([32, 32])
blocks.0.sa.heads.3.key.weight 	 torch.Size([16, 64])
blocks.0.sa.heads.3.query.weight 	 torch.Size([16, 64])
blocks.0.sa.heads.3.value.weight 	 torch.Size([16, 64])
blocks.0.sa.proj.

In [26]:
torch.save(model.state_dict(), 'model.pt')

In [47]:
model = BigramLanguageModel()
model.load_state_dict(torch.load('model.pt'))
model.train()

BigramLanguageModel(
  (token_embedding_table): Embedding(83, 64)
  (position_embedding_table): Embedding(32, 64)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x Head(
            (key): Linear(in_features=64, out_features=16, bias=False)
            (query): Linear(in_features=64, out_features=16, bias=False)
            (value): Linear(in_features=64, out_features=16, bias=False)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (proj): Linear(in_features=64, out_features=64, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=64, out_features=256, bias=True)
          (1): ReLU()
          (2): Linear(in_features=256, out_features=64, bias=True)
          (3): Dropout(p=0.0, inplace=False)
        )
      )
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    

In [48]:
from transformers import TrainingArguments
import os

# os.makedirs("test_trainer", exist_ok=True)


training_args = TrainingArguments(output_dir="test_trainer", per_device_train_batch_size=16, per_device_eval_batch_size=16, num_train_epochs=3, evaluation_strategy="epoch", logging_dir="test_trainer/logs", logging_strategy="epoch")

In [49]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


In [50]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [51]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [52]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [53]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [56]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [57]:
trainer.train()

TypeError: forward() got an unexpected keyword argument 'labels'

In [41]:
train_data,val_data

(tensor([73, 77, 61,  ..., 74, 65, 72]),
 tensor([76, 65, 71,  ..., 19,  0,  0]))

In [58]:
train_data

tensor([73, 77, 61,  ..., 74, 65, 72])

In [46]:
small_eval_dataset['label']

[2,
 4,
 1,
 4,
 3,
 4,
 2,
 3,
 2,
 3,
 0,
 0,
 3,
 2,
 2,
 1,
 3,
 1,
 2,
 2,
 1,
 2,
 3,
 1,
 1,
 3,
 4,
 0,
 0,
 2,
 2,
 2,
 1,
 3,
 4,
 0,
 0,
 1,
 3,
 2,
 0,
 2,
 0,
 0,
 3,
 0,
 3,
 2,
 3,
 0,
 1,
 1,
 3,
 3,
 4,
 4,
 1,
 4,
 1,
 3,
 1,
 0,
 0,
 1,
 4,
 1,
 4,
 3,
 2,
 4,
 1,
 0,
 3,
 3,
 4,
 1,
 2,
 1,
 0,
 4,
 4,
 4,
 2,
 3,
 3,
 1,
 4,
 0,
 4,
 2,
 3,
 0,
 0,
 0,
 3,
 4,
 0,
 0,
 1,
 4,
 4,
 0,
 0,
 1,
 1,
 0,
 4,
 2,
 2,
 1,
 1,
 4,
 0,
 4,
 0,
 3,
 2,
 0,
 4,
 4,
 4,
 2,
 0,
 0,
 0,
 1,
 3,
 0,
 2,
 0,
 3,
 2,
 2,
 2,
 0,
 3,
 4,
 3,
 0,
 1,
 0,
 1,
 0,
 0,
 4,
 3,
 3,
 1,
 3,
 0,
 3,
 4,
 0,
 2,
 1,
 3,
 1,
 3,
 1,
 3,
 1,
 0,
 2,
 0,
 1,
 1,
 0,
 0,
 3,
 3,
 3,
 3,
 3,
 1,
 1,
 0,
 0,
 2,
 3,
 3,
 3,
 1,
 1,
 3,
 4,
 0,
 1,
 1,
 2,
 2,
 1,
 3,
 0,
 2,
 1,
 1,
 4,
 0,
 2,
 4,
 1,
 1,
 2,
 3,
 3,
 2,
 0,
 1,
 2,
 4,
 4,
 1,
 4,
 2,
 3,
 0,
 1,
 0,
 0,
 2,
 4,
 3,
 3,
 0,
 1,
 1,
 2,
 0,
 4,
 0,
 3,
 0,
 3,
 2,
 3,
 1,
 2,
 4,
 4,
 2,
 1,
 0,
 3,
 1,
 1,
 1,
 3,
 0,
 3,
 0,


In [42]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("squad")
dataset["train"][1]


{'id': '5733be284776f4190066117f',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'What is in front of the Notre Dame Main Building?',
 'answers': {'text': ['a copper statue of Christ'], 'answer_start': [188]}}

TypeError: 'Dataset' object does not support item assignment

In [61]:
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, Trainer, TrainingArguments
import tiktoken
from datasets import load_dataset

model_name = "distilbert-base-cased-distilled-squad"  # Adjust as needed
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)


# with open("/Users/jagpreetsingh/ML_Projects/text-sql/spider_train.txt", "r", encoding="utf-8") as f:
#     train = f.read()
# with open("/Users/jagpreetsingh/ML_Projects/text-sql/spider_test.txt", "r", encoding="utf-8") as f:
#     test = f.read()

# data_files = {"train": train, "test": test}


# tokenized_datasets = tokenize_function(dataset, split="train")

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

print("sample train", small_train_dataset[0])
# train_dataset = tokenizer(train, padding="do_not_pad", truncation=True)
# eval_dataset = tokenizer(test, padding="do_not_pad", truncation=True)

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
)
trainer.train()


tokenizer_config.json: 100%|██████████| 29.0/29.0 [00:00<00:00, 12.4kB/s]
config.json: 100%|██████████| 473/473 [00:00<00:00, 380kB/s]
vocab.txt: 100%|██████████| 213k/213k [00:00<00:00, 3.55MB/s]
tokenizer.json: 100%|██████████| 436k/436k [00:00<00:00, 7.13MB/s]
model.safetensors: 100%|██████████| 261M/261M [00:03<00:00, 80.8MB/s]
Map:   0%|          | 0/87599 [00:00<?, ? examples/s]


KeyError: 'text'